In [1]:
# import everything as needed
%matplotlib inline
# %matplotlib notebook

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import math
import json
import re
import string
from matplotlib.artist import setp

from operator import itemgetter

from irkit.trec import qrels
from irkit.trec import results
from irkit.trec import run
from irkit.query import elasticsearch as eq

from elasticsearch import Elasticsearch
from elasticsearch import client

PORT = 9200
INDEX = "ussc"

es = Elasticsearch(urls='http://localhost', port=PORT, timeout=600)
es_client = client.IndicesClient(es)

from IPython.display import set_matplotlib_formats
# To get all the plots in pdf, change this to 'pdf'!
set_matplotlib_formats('pdf')

# Set general plot properties.
sns.set()
sns.set_context("paper")
sns.set_color_codes("pastel")

sns.set_context({"figure.figsize": (16, 10)})
plt.style.use('grayscale')
plt.style.use('seaborn-white')

First of all, we need to extract and transform data from the topic.

In [2]:
def remove_stop(text):
    a_bod = {
    "analyzer": "stop",
    "text": text
    }
    final_text = ""
    res = es_client.analyze(index=INDEX, body=a_bod)
    for i in range(len(res["tokens"])):
        final_text = final_text + " " + res["tokens"][i]["token"]
    return final_text

def clean_text(text):
    return re.sub('[^a-zA-Z0-9]+', ' ', text).lower()

def get_numbers(text):
    return re.sub('[^0-9]+', ' ', text).lstrip().rstrip()

topics = {}
for i in range(1, 101):
    with open('../../airs2017-collection/topic/{}.json'.format(i)) as f:
        topic = json.load(f)

    queries = topic['es_query'] # Elasticsearch DSL query
    relevant_keywords = topic['relevant_keywords'] # Relevant keywords query
    
    citing_sentence = topic['citing_sentence'] # Citing sentence (plain text)
    citing_paragraph = topic['citing_paragraph'] # Citing paragrah (plain text)
    case_extract = topic['case_extract'] # Sentences removed from the citing sentence

    # Remove citing sentences.
    for sentence in case_extract:
        citing_sentence = citing_sentence.replace(sentence, '')
        citing_paragraph = citing_paragraph.replace(sentence, '')

    # !! Add `0-9` back in to re-introduce numbers !!
    numbers = get_numbers(citing_sentence)
    citing_sentence = clean_text(citing_sentence)
    citing_sentence = remove_stop(citing_sentence)
    citing_sentence = citing_sentence + " " + numbers
    sent_tokens = set(citing_sentence.split())
    
    numbers = get_numbers(citing_paragraph)
    citing_paragraph = clean_text(citing_paragraph)
    citing_paragraph = remove_stop(citing_paragraph)
    citing_paragraph = citing_paragraph + " " + numbers
    para_tokens = set(citing_paragraph.split())

    # Tokenise and remove stopwords.
    keywords = " ".join(relevant_keywords)
    keywords = clean_text(keywords)
    keywords = set(keywords.split())
    relevant_tokens = list(keywords)

    topics[topic['id']] = {'queries': queries, 
                           'sentence_tokens': sent_tokens, 
                           'paragraph_tokens': para_tokens,
                          'keyword_tokens': relevant_tokens}

Once we have the tokens, we can work out extracting the keywords from both the manual and automatic queries. For the manual queries, we will create a visitor to find the leaf nodes in the query DSL and add them into our existing data struture. For the automatic queries, we can use the intermediate representation to do so.

In [3]:
keys = ['idfp_auto_query_terms', 'idfs_auto_query_terms', 'klis_auto_query_terms', 'klip_auto_query_terms',
        'plms_5_auto_query_terms','plmp_5_auto_query_terms']

class MultiMatchAppenderVisitor(eq.Visitor):
    def __init__(self, node_name):
        super().__init__(node_name)
        self.result = []
    
    def visit(self, node):
        self.result.append(node[self.node_name]['query'])

# Process the manual query.
for topic_id, topic_data in topics.items():
    # Use the visitor to extract the query terms.
    query_terms = set()
    queries = []
    for query in topic_data['queries']:
        query_terms = query_terms.union(set(eq.traverse(query, MultiMatchAppenderVisitor('multi_match'))))
        queries.append(list(set([i for s in [re.sub('[^a-zA-Z0-9]+', ' ', x).split() for x in list(query_terms)] for i in s])))
    # Process the query terms to make them tokens.
    topic_data['query_tokens'] = queries
    
# Process each `automatic` query.
for i in range(1, 101):
    for t in keys:
        with open('../auto-generation/queries/{}_{}.json'.format(i, t), 'r') as f:
            query = json.load(f)
            queries = query['auto_generated_queries']
            
            topics[queries['topic']][t] = queries['query']

Now that we have extracted the tokens from both the the manually constructed queries and the automatically generated ones, we can perform the analysis.

In [4]:
def calculate_overlap(tokens, sentence_tokens):
    """ 
    The proportion of the number of tokens in the query that also are in the sentence, 
    divided by the number of tokens in the sentence
    """
    return len(set(tokens).intersection(set(sentence_tokens)))/len(sentence_tokens)

In [5]:
def check_methods():
    for i in range(1, 101):
        for j in range(0, 11):
            
            idfp = calculate_overlap(topics[i]['idfp_auto_query_terms'][j]['terms'], topics[i]['paragraph_tokens'])
            plmp = calculate_overlap(topics[i]['plmp_5_auto_query_terms'][j]['terms'], topics[i]['paragraph_tokens'])
            klip = calculate_overlap(topics[i]['klip_auto_query_terms'][j]['terms'], topics[i]['paragraph_tokens'])

            idfs = calculate_overlap(topics[i]['idfs_auto_query_terms'][j]['terms'], topics[i]['sentence_tokens'])
            plms = calculate_overlap(topics[i]['plms_5_auto_query_terms'][j]['terms'], topics[i]['sentence_tokens'])
            klis = calculate_overlap(topics[i]['klis_auto_query_terms'][j]['terms'], topics[i]['sentence_tokens'])
            
            if idfp != plmp or plmp != klip or idfp != klip:
                print("*"*30)
                print("Para mismatch")
                print("Topic -", i, "- proportion -", j)
                print(idfp, plmp, klip)
                print(len(topics[i]['idfp_auto_query_terms'][j]['terms']), 
                      len(topics[i]['plmp_5_auto_query_terms'][j]['terms']),
                      len(topics[i]['klip_auto_query_terms'][j]['terms']))
                print("Difference -", set(topics[i]['paragraph_tokens']).difference(set(topics[i]['idfp_auto_query_terms'][j]['terms'])))
                print("Difference -", set(topics[i]['idfp_auto_query_terms'][j]['terms']).difference(set(topics[i]['klip_auto_query_terms'][j]['terms'])))
            
            if idfs != plms or plms != klis or idfs != klis:
                print("*"*30)
                print("Topic -", i, "- proportion -", j)
                print(idfs, plms, klis)

check_methods()

In [6]:
def fill_empty_row(df):
    topic_not_found = set()
    for i in range(1, 101):
        topic_not_found.add(i)

    for row in df.iterrows():
        topic_not_found.remove(row[0])
        
    num_colls = len(df.columns)
    
    for row in topic_not_found: 
        df.loc[row] = (0) * num_colls

        
#find the file that contains the best AP_5 score for the topic
def get_best_run():
    best_runs = {}
    for i in range(1, 101):
        best_runs[i] = {'score': 0, 'query': 0}
    
    runs = range(1, 4)
    for run in runs:  
        with open('res-files/{}{}_standard.csv'.format("adhocbool", run), 'r') as f:
            df = pd.DataFrame.from_csv(f)
            fill_empty_row(df)
            for y in df.iterrows():
                if best_runs[y[0]]['score'] < y[1][' map']:
                    best_runs[y[0]]['score'] = y[1][' map']
                    best_runs[y[0]]['query'] = run - 1
                    
    return best_runs


                    

In [7]:
keys = ['idfs_auto_query_terms', 'idfp_auto_query_terms', 'klis_auto_query_terms', 'klip_auto_query_terms',
        'plms_5_auto_query_terms','plmp_5_auto_query_terms']
run_names = ["idfs","idfp","klis","klip","plms","plmp"]
     

In [8]:
query_tokens = {}
def create_bool_overlap(bar):
    nm = 1
    for r in bar: 
        print(r)
        x = []
        for i in range(0,11):
            x_i = []
            for k, v in get_best_run().items():   
                x_i.append(calculate_overlap(topics[k][r][i]['terms'], topics[k]['query_tokens'][v['query']]))
               
            x.append(np.mean(x_i))
        plt.plot(np.arange(0.0, 1.1, 0.1), x, linewidth=4, marker='o', markersize=6, label=run_names[nm - 1], linestyle='--', dashes=(nm,1))
        
        plt.xlabel("$r$",fontsize=30)
        plt.ylabel("Overlap",fontsize=30)
        plt.ylim((0, 0.6))
        # plt.xlim((0, 1.0))
        plt.tick_params(labelsize=20)
        plt.legend(fontsize=18)
        nm += 1
    plt.show()

def create_key_overlap(runs):
    nm = 1
    for r in runs:
        print(r)  
        x = []
        for i in range(0,11):
            x_i = []
            for k, v in get_best_run().items():
                x_i.append(calculate_overlap(topics[k][r][i]['terms'], topics[k]['keyword_tokens']))
            x.append(np.mean(x_i))
        
        plt.plot(np.arange(0.0, 1.1, 0.1), x, linewidth=4, marker='o', markersize=6, label=run_names[nm - 1], linestyle='--', dashes=(nm,1))
        plt.xlabel("$r$",fontsize=30)
        plt.ylabel("Overlap",fontsize=30)
        plt.ylim((0, 0.6))
        # ax.axes.get_xaxis().set_ticks([])
        plt.tick_params(labelsize=20)
        plt.legend(fontsize=18)
        nm += 1
    plt.show()

In [9]:
create_bool_overlap(keys)
create_key_overlap(keys)

idfs_auto_query_terms
idfp_auto_query_terms
klis_auto_query_terms
klip_auto_query_terms
plms_5_auto_query_terms
plmp_5_auto_query_terms


idfs_auto_query_terms
idfp_auto_query_terms
klis_auto_query_terms
klip_auto_query_terms
plms_5_auto_query_terms
plmp_5_auto_query_terms
